In [ ]:
import h5py

f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())
f.close()

In [ ]:
f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'r')
print(dir(f))
dset = f['coords']
last_points = dset[-2:]

In [ ]:
import numpy as np
import cv2

In [ ]:
f = h5py.File('/home/shifa/AgentHuman/SeqTrain/data_03663.h5', 'r')
imgs = np.array(f['rgb'])
print(imgs.shape)
targets = np.array(f['targets'])

In [ ]:
img = imgs[100]
target = targets[100]
target = target.astype(np.float32)
print(target.shape)
print(target)
# 2 Follow lane, 3 Left, 4 Right, 5 Straight
# -> 0 Follow lane, 1 Left, 2 Right, 3 Straight
command = int(target[24])-2
print('command', command)
# Steer, Gas, Brake (0,1, focus on steer loss)
target_vec = np.zeros((4, 3), dtype=np.float32)
target_vec[command, :] = target[:3]
print(target_vec)
            # in km/h, <90
speed = np.array([target[10]/90, ]).astype(np.float32)
print('speed', speed)
mask_vec = np.zeros((4, 3), dtype=np.float32)
mask_vec[command, :] = 1
print(mask_vec.shape)
plt.imshow(img)
plt.show()
tar = target_vec.reshape(-1)
mask = mask_vec.reshape(-1)
print(speed, tar, mask)

In [ ]:
idx = 0
while(idx<200):
    frame = imgs[idx]
    idx += 1
    cv2.imshow('frame',frame)
    k = cv2.waitKey(100) & 0xff
    if k == 27:
        break
cv2.destroyAllWindows()